Import from external sources:

In [3]:
import pandas as pd
from ema_workbench import (Model, CategoricalParameter,
                           ScalarOutcome, IntegerParameter, RealParameter)
from dike_model_function import DikeNetwork  # @UnresolvedImport

from ema_workbench import (Model, MultiprocessingEvaluator, Policy, Scenario)

from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.util import ema_logging
import time

Define methods:
Code modified from <insert original file name here>:

In [ ]:
def sum_over(*args):
    return sum(args)

def get_model_for_problem_formulation(problem_formulation_id):
    ''' Prepare DikeNetwork in a way it can be input in the EMA-workbench.
    Specify uncertainties, levers and problem formulation.
    '''
    # Load the model:
    function = DikeNetwork()
    # workbench model:
    dike_model = Model('dikesnet', function=function)

    # Uncertainties and Levers:
    # Specify uncertainties range:
    Real_uncert = {'Bmax': [30, 350], 'pfail': [0, 1]}  # m and [.]
    # breach growth rate [m/day]
    cat_uncert_loc = {'Brate': (1., 1.5, 10)}

    cat_uncert = {'discount rate {}'.format(n): (1.5, 2.5, 3.5, 4.5)
                  for n in function.planning_steps}

    Int_uncert = {'A.0_ID flood wave shape': [0, 132]}
    # Range of dike heightening:
    dike_lev = {'DikeIncrease': [0, 10]}    # dm

    # Series of five Room for the River projects:
    rfr_lev = ['{}_RfR'.format(project_id) for project_id in range(0, 5)]

    # Time of warning: 0, 1, 2, 3, 4 days ahead from the flood
    EWS_lev = {'EWS_DaysToThreat': [0, 4]}  # days

    uncertainties = []
    levers = []

    for uncert_name in cat_uncert.keys():
        categories = cat_uncert[uncert_name]
        uncertainties.append(CategoricalParameter(uncert_name, categories))

    for uncert_name in Int_uncert.keys():
        uncertainties.append(IntegerParameter(uncert_name,
                                              Int_uncert[uncert_name][0],
                                              Int_uncert[uncert_name][1]))

        # RfR levers can be either 0 (not implemented) or 1 (implemented)
    for lev_name in rfr_lev:
        for n in function.planning_steps:
            lev_name_ = '{} {}'.format(lev_name, n)
            levers.append(IntegerParameter(lev_name_, 0, 1))

    # Early Warning System lever
    for lev_name in EWS_lev.keys():
        levers.append(IntegerParameter(lev_name, EWS_lev[lev_name][0],
                                       EWS_lev[lev_name][1]))

    for dike in function.dikelist:
        # uncertainties in the form: locationName_uncertaintyName
        for uncert_name in Real_uncert.keys():
            name = "{}_{}".format(dike, uncert_name)
            lower, upper = Real_uncert[uncert_name]
            uncertainties.append(RealParameter(name, lower, upper))

        for uncert_name in cat_uncert_loc.keys():
            name = "{}_{}".format(dike, uncert_name)
            categories = cat_uncert_loc[uncert_name]
            uncertainties.append(CategoricalParameter(name, categories))

        # location-related levers in the form: locationName_leversName
        for lev_name in dike_lev.keys():
            for n in function.planning_steps:
                name = "{}_{} {}".format(dike, lev_name, n)
                levers.append(IntegerParameter(name, dike_lev[lev_name][0],
                                               dike_lev[lev_name][1]))

    # load uncertainties and levers in dike_model:
    dike_model.uncertainties = uncertainties
    dike_model.levers = levers

    # Problem formulations:
    # Outcomes are all costs, thus they have to minimized:
    direction = ScalarOutcome.MINIMIZE

    # 2-objective PF:
    if problem_formulation_id == 0:
        variable_names = []
        variable_names_ = []

        for n in function.planning_steps:

            variable_names.extend(
                ['{}_{} {}'.format(dike, e, n) for e in [
                    'Expected Annual Damage', 'Dike Investment Costs'] for dike in function.dikelist])

            variable_names_.extend(
                ['{}_{} {}'.format(dike, e, n) for e in [
                    'Expected Number of Deaths'] for dike in function.dikelist])

            variable_names.extend(['RfR Total Costs {}'.format(n)])
            variable_names.extend(['Expected Evacuation Costs {}'.format(n)])

        dike_model.outcomes = [ScalarOutcome('All Costs',
                                             variable_name=[
                                                 var for var in variable_names],
                                             function=sum_over, kind=direction),

                               ScalarOutcome('Expected Number of Deaths',
                                             variable_name=[var for var in variable_names_
                                                            ], function=sum_over, kind=direction)]

    # 3-objectives PF:
    elif problem_formulation_id == 1:
        variable_names = []
        variable_names_ = []
        variable_names__ = []

        for n in function.planning_steps:
            variable_names.extend(['{}_Expected Annual Damage {}'.format(dike, n)
                                   for dike in function.dikelist])

            variable_names_.extend(['{}_Dike Investment Costs {}'.format(dike, n)
                                    for dike in function.dikelist] + [
                                       'RfR Total Costs {}'.format(n)
                                   ] + ['Expected Evacuation Costs {}'.format(n)])

            variable_names__.extend(['{}_Expected Number of Deaths {}'.format(dike, n)
                                     for dike in function.dikelist])


        dike_model.outcomes = [
            ScalarOutcome('Expected Annual Damage',
                          variable_name=[var for var in variable_names],
                          function=sum_over, kind=direction),

            ScalarOutcome('Total Investment Costs',
                          variable_name=[var for var in variable_names_],
                          function=sum_over, kind=direction),

            ScalarOutcome('Expected Number of Deaths',
                          variable_name=[var for var in variable_names__],
                          function=sum_over, kind=direction)]

    # 5-objectives PF:
    elif problem_formulation_id == 2:
        variable_names = []
        variable_names_ = []
        variable_names__ = []
        variable_names___ = []
        variable_names____ = []

        for n in function.planning_steps:
            variable_names.extend(['{}_Expected Annual Damage {}'.format(dike, n)
                                   for dike in function.dikelist])
            variable_names_.extend(['{}_Dike Investment Costs {}'.format(dike, n)
                                    for dike in function.dikelist])
            variable_names__.extend(['RfR Total Costs {}'.format(n)])
            variable_names___.extend(['Expected Evacuation Costs {}'.format(n)])
            variable_names____.extend(['{}_Expected Number of Deaths {}'.format(dike, n)
                                       for dike in function.dikelist])

        dike_model.outcomes = [
            ScalarOutcome('Expected Annual Damage',
                          variable_name=[var for var in variable_names],
                          function=sum_over, kind=direction),

            ScalarOutcome('Dike Investment Costs',
                          variable_name=[var for var in variable_names_],
                          function=sum_over, kind=direction),

            ScalarOutcome('RfR Investment Costs',
                          variable_name=[var for var in variable_names__],
                          function=sum_over, kind=direction),

            ScalarOutcome('Evacuation Costs',
                          variable_name=[var for var in variable_names___],
                          function=sum_over, kind=direction),

            ScalarOutcome('Expected Number of Deaths',
                          variable_name=[var for var in variable_names____],
                          function=sum_over, kind=direction)]

    # Disaggregate over locations:
    elif problem_formulation_id == 3:
        outcomes = []

        for dike in function.dikelist:
            variable_name = []
            for e in ['Expected Annual Damage', 'Dike Investment Costs']:
                variable_name.extend(['{}_{} {}'.format(dike, e, n)
                                      for n in function.planning_steps])

            outcomes.append(ScalarOutcome('{} Total Costs'.format(dike),
                                          variable_name=[var for var in variable_name],
                                          function=sum_over, kind=direction))

            outcomes.append(ScalarOutcome('{}_Expected Number of Deaths'.format(dike),
                                          variable_name=['{}_Expected Number of Deaths {}'.format(
                                              dike, n) for n in function.planning_steps],
                                          function=sum_over, kind=direction))

        outcomes.append(ScalarOutcome('RfR Total Costs',
                                      variable_name=['RfR Total Costs {}'.format(n
                                                                                 ) for n in function.planning_steps],
                                      function=sum_over, kind=direction))
        outcomes.append(ScalarOutcome('Expected Evacuation Costs',
                                      variable_name=['Expected Evacuation Costs {}'.format(n
                                                                                           ) for n in function.planning_steps],
                                      function=sum_over, kind=direction))

        dike_model.outcomes = outcomes

    # Disaggregate over time:
    elif problem_formulation_id == 4:
        outcomes = []

        for n in function.planning_steps:
            for dike in function.dikelist:

                outcomes.append(ScalarOutcome('Expected Annual Damage {}'.format(n),
                                              variable_name=['{}_Expected Annual Damage {}'.format(dike,n)
                                                             for dike in function.dikelist],
                                              function=sum_over, kind=direction))

                outcomes.append(ScalarOutcome('Dike Investment Costs {}'.format(n),
                                              variable_name=['{}_Dike Investment Costs {}'.format(dike,n)
                                                             for dike in function.dikelist],
                                              function=sum_over, kind=direction))

                outcomes.append(ScalarOutcome('Expected Number of Deaths {}'.format(n),
                                              variable_name=['{}_Expected Number of Deaths {}'.format(dike,n)
                                                             for dike in function.dikelist],
                                              function=sum_over, kind=direction))

            outcomes.append(ScalarOutcome('RfR Total Costs {}'.format(n),
                                          kind=direction))
            outcomes.append(ScalarOutcome('Expected Evacuation Costs {}'.format(n),
                                          kind=direction))

        dike_model.outcomes = outcomes

    # Fully disaggregated:
    elif problem_formulation_id == 5:
        outcomes = []

        for n in function.planning_steps:
            for dike in function.dikelist:
                for entry in ['Expected Annual Damage', 'Dike Investment Costs',
                              'Expected Number of Deaths']:

                    o = ScalarOutcome('{}_{} {}'.format(dike, entry, n), kind=direction)
                    outcomes.append(o)

            outcomes.append(ScalarOutcome('RfR Total Costs {}'.format(n), kind=direction))
            outcomes.append(ScalarOutcome('Expected Evacuation Costs {}'.format(n), kind=direction))
        dike_model.outcomes = outcomes

    else:
        raise TypeError('unknownx identifier')

    return dike_model, function.planning_steps

Code modified from <insert original file name here>:

In [5]:
ema_logging.log_to_stderr(ema_logging.INFO)

#choose problem formulation number, between 0-5
#each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(3)

#enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary

uncertainties = dike_model.uncertainties

import copy
uncertainties = copy.deepcopy(dike_model.uncertainties)

#enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary

levers = dike_model.levers

import copy
levers = copy.deepcopy(dike_model.levers)


#running the model through EMA workbench
from ema_workbench import (MultiprocessingEvaluator, ema_logging,
                           perform_experiments, SequentialEvaluator)
ema_logging.log_to_stderr(ema_logging.INFO)

with SequentialEvaluator(dike_model) as evaluator:
    results = evaluator.perform_experiments(scenarios=5, policies=2)

[MainProcess/INFO] performing 5 scenarios * 2 policies * 1 model(s) = 10 experiments
100%|██████████████████████████████████████████| 10/10 [00:17<00:00,  1.72s/it]
[MainProcess/INFO] experiments finished


In [6]:
# save outcomes somewhere
experiments, outcomes = results
#outcomes_csv = pd.DataFrame.from_dict(outcomes)

experiments.to_csv('./data/test_experiments.csv', index=False)
pd.DataFrame.from_dict(outcomes).to_csv('./data/test_outcomes.csv', index=False)

In [7]:
experiments

,A.0_ID flood wave shape,A.1_Bmax,A.1_Brate,A.1_pfail,A.2_Bmax,A.2_Brate,A.2_pfail,A.3_Bmax,A.3_Brate,A.3_pfail,...,A.4_DikeIncrease 0,A.4_DikeIncrease 1,A.4_DikeIncrease 2,A.5_DikeIncrease 0,A.5_DikeIncrease 1,A.5_DikeIncrease 2,EWS_DaysToThreat,scenario,policy,model
0,31,93.205532,10,0.020796,291.461208,1.0,0.287034,340.262830,10,0.005467,...,7,9,0,2,4,6,3,2,0,dikesnet
1,25,222.602968,1.0,0.376424,203.537739,1.5,0.718330,234.800911,1.5,0.609713,...,7,9,0,2,4,6,3,3,0,dikesnet
2,110,151.868974,10,0.962456,48.627460,1.0,0.868484,123.802512,10,0.353854,...,7,9,0,2,4,6,3,4,0,dikesnet
3,102,205.095009,1.0,0.493941,281.572119,10,0.065085,206.360646,1.5,0.847299,...,7,9,0,2,4,6,3,5,0,dikesnet
4,72,346.238369,1.5,0.760720,111.730620,10,0.457586,50.784108,1.0,0.551831,...,7,9,0,2,4,6,3,6,0,dikesnet
5,31,93.205532,10,0.020796,291.461208,1.0,0.287034,340.262830,10,0.005467,...,3,2,6,10,10,1,0,2,1,dikesnet
6,25,222.602968,1.0,0.376424,203.537739,1.5,0.718330,234.800911,1.5,0.609713,...,3,2,6,10,10,1,0,3,1,dikesnet
7,110,151.868974,10,0.962456,48.627460,1.0,0.868484,123.802512,10,0.353854,...,3,2,6,10,10,1,0,4,1,dikesnet
8,102,205.095009,1.0,0.493941,281.572119,10,0.065085,206.360646,1.5,0.847299,...,3,2,6,10,10,1,0,5,1,dikesnet
9,72,346.238369,1.5,0.760720,111.730620,10,0.457586,50.784108,1.0,0.551831,...,3,2,6,10,10,1,0,6,1,dikesnet


In [8]:
pd.DataFrame.from_dict(outcomes)

,A.1 Total Costs,A.1_Expected Number of Deaths,A.2 Total Costs,A.2_Expected Number of Deaths,A.3 Total Costs,A.3_Expected Number of Deaths,A.4 Total Costs,A.4_Expected Number of Deaths,A.5 Total Costs,A.5_Expected Number of Deaths,RfR Total Costs,Expected Evacuation Costs
0,2.248880e+09,0.161398,2.677779e+08,0.000000,3.817946e+08,0.054203,3.721685e+07,0.000000,1.125445e+08,0.000000,6.641000e+08,37363.396356
1,2.020228e+08,0.005703,2.677779e+08,0.000000,9.373029e+07,0.000000,3.721685e+07,0.000000,1.699040e+08,0.005959,6.641000e+08,4593.496335
2,1.314447e+08,0.000000,2.677779e+08,0.000000,9.525743e+07,0.000498,3.721685e+07,0.000000,1.145980e+08,0.000341,6.641000e+08,289.474714
3,1.394526e+08,0.000846,2.732990e+08,0.000775,9.373029e+07,0.000000,3.721685e+07,0.000000,1.125445e+08,0.000000,6.641000e+08,534.208998
4,1.314447e+08,0.000000,2.677779e+08,0.000000,9.373029e+07,0.000000,3.721685e+07,0.000000,1.332762e+08,0.001721,6.641000e+08,973.728983
5,2.039039e+08,0.000000,1.379726e+08,0.012076,1.378038e+08,0.000000,3.145208e+07,0.000000,1.886121e+08,0.000000,1.467100e+09,0.000000
6,2.039039e+08,0.000000,1.260496e+08,0.001153,1.378038e+08,0.000000,3.145208e+07,0.000000,1.886121e+08,0.000000,1.467100e+09,0.000000
7,2.039039e+08,0.000000,1.248279e+08,0.000000,1.378038e+08,0.000000,3.145208e+07,0.000000,1.886121e+08,0.000000,1.467100e+09,0.000000
8,2.039039e+08,0.000000,2.085689e+08,0.090929,1.378038e+08,0.000000,3.145208e+07,0.000000,1.886121e+08,0.000000,1.467100e+09,0.000000
9,2.039039e+08,0.000000,1.306439e+08,0.004173,1.378038e+08,0.000000,3.807441e+07,0.002571,1.886121e+08,0.000000,1.467100e+09,0.000000
